In [22]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

## Extraction

* creating a file path
* reading file in pandas

In [23]:
#Extract

intake_file = "Resources/austin-animal-center-intakes.csv"
outcome_file = "Resources/austin-animal-center-outcomes.csv"

intake_df = pd.read_csv(intake_file)
outcome_df = pd.read_csv(outcome_file)



In [24]:
#preview outcomes
outcome_df.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A771830,*Bradley,12/27/2018 01:25:00 PM,12/27/2018 01:25:00 PM,05/10/2017,Adoption,Foster,Dog,Neutered Male,1 year,Pit Bull Mix,White/Blue
1,A779576,*Rajah,10/01/2018 05:03:00 PM,10/01/2018 05:03:00 PM,04/18/2018,Adoption,NaN,Cat,Neutered Male,5 months,Domestic Shorthair Mix,Brown Tabby
2,A741715,*Pebbles,01/11/2017 06:17:00 PM,01/11/2017 06:17:00 PM,03/07/2016,Adoption,NaN,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Calico
3,A658751,Benji,11/13/2016 01:38:00 PM,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,NaN,Dog,Neutered Male,5 years,Border Terrier Mix,Tan
4,A721285,NaN,02/24/2016 02:42:00 PM,02/24/2016 02:42:00 PM,02/24/2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon Mix,Black/Gray


In [25]:
#preview outcome
intake_df.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A748291,*Madison,05/01/2017 02:26:00 PM,05/01/2017 02:26:00 PM,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,Black
1,A750529,NaN,05/28/2017 01:22:00 PM,05/28/2017 01:22:00 PM,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,07/07/2016 12:11:00 PM,07/07/2016 12:11:00 PM,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,05/01/2017 10:53:00 AM,05/01/2017 10:53:00 AM,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,07/13/2014 11:02:00 AM,07/13/2014 11:02:00 AM,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,Brown/White


## Transforamtion
* reduce to pertinent columns
* reformat 'DateTime' to reflect needed information
* rename columns for sql

In [26]:
#Viewing columns needed
intake_cleaned_df = intake_df[["Animal ID","Name","DateTime","Intake Type","Intake Condition","Age upon Intake","Breed","Color"]]

intake_cleaned_df.head()

,Animal ID,Name,DateTime,Intake Type,Intake Condition,Age upon Intake,Breed,Color
0,A748291,*Madison,05/01/2017 02:26:00 PM,Stray,Normal,10 months,Pit Bull Mix,Black
1,A750529,NaN,05/28/2017 01:22:00 PM,Stray,Normal,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,07/07/2016 12:11:00 PM,Stray,Normal,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,05/01/2017 10:53:00 AM,Stray,Normal,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,07/13/2014 11:02:00 AM,Owner Surrender,Nursing,4 weeks,Border Collie Mix,Brown/White


In [27]:
#reformat date (stripped of the timestamp)
intake_cleaned_df = intake_cleaned_df.copy()
intake_cleaned_df["DateTime"] = pd.to_datetime(intake_cleaned_df["DateTime"]).dt.date
intake_cleaned_df.head()


,Animal ID,Name,DateTime,Intake Type,Intake Condition,Age upon Intake,Breed,Color
0,A748291,*Madison,2017-05-01,Stray,Normal,10 months,Pit Bull Mix,Black
1,A750529,NaN,2017-05-28,Stray,Normal,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,2016-07-07,Stray,Normal,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,2017-05-01,Stray,Normal,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,2014-07-13,Owner Surrender,Nursing,4 weeks,Border Collie Mix,Brown/White


In [28]:
intake_table = intake_cleaned_df
intake_table = intake_table.rename(columns = {"Animal ID":"Animal_ID",
                                                  "Intake Type": "Intake_Type",
                                                  "Intake Condition" : "Intake_Condition",
                                                  "Age upon Intake" : "Age"})
intake_table.head()

,Animal_ID,Name,DateTime,Intake_Type,Intake_Condition,Age,Breed,Color
0,A748291,*Madison,2017-05-01,Stray,Normal,10 months,Pit Bull Mix,Black
1,A750529,NaN,2017-05-28,Stray,Normal,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,2016-07-07,Stray,Normal,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,2017-05-01,Stray,Normal,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,2014-07-13,Owner Surrender,Nursing,4 weeks,Border Collie Mix,Brown/White


## Load

* create database connection
* loaded dateframes in sql


In [35]:
rd_connection_string = "postgres:password@localhost:5432/austin_animal_center"
engine = create_engine(f"postgresql://{rd_connection_string}")

In [37]:
#test is connection to db was successful
engine.table_names()

['intakes']

In [ ]:
#FOR MERGE
#list of common columns

common_columns = ["Animal ID","Name","DateTime","Breed","Color"]
